In [ ]:
# Ensure only one CPU is used:
import os
os.environ["OMP_NUM_THREADS"] = "1"

import sys
sys.path.append('..')

from pathlib import Path
import pandas as pd

from cogwheel import posterior
from cogwheel import sampling
from cogwheel import data
from cogwheel import likelihood
from cogwheel import gw_plotting

In [ ]:
parentdir = Path('example')  # PE runs will be saved here

In [ ]:
eventname = 'GW190412'

In [ ]:
%%time
# Find reference waveform and compute summary data:
post = posterior.Posterior.from_event(
    eventname,
    data.EVENTS_METADATA['mchirp'][eventname],
    'IMRPhenomXPHM',
    prior_class='IntrinsicIASPrior',
    likelihood_class=likelihood.MarginalizedExtrinsicLikelihood)

In [ ]:
%%time
# Run sampler:
pym = sampling.PyMultiNest(post)
pym.run_kwargs['n_live_points'] = 128
rundir = pym.get_rundir(parentdir)
pym.run(rundir)

In [ ]:
samples = pd.read_feather(rundir/sampling.SAMPLES_FILENAME)

params = [
    'mchirp',
    'lnq',
    'chieff',
    'cumchidiff',
    'costheta_jn',
    'phi_jl_hat',
    'phi12',
    'cums1r_s1z',
    'cums2r_s2z',
    'd_luminosity',
    'dec',
    'phi_ref',
    'psi',
    't_geocenter',
    'lnl_marginalized',
    'lnl',
    'ra'
]

gw_plotting.CornerPlot(samples[params]).plot()